In [1]:
import pandas as pd
import requests

In [6]:
#get text from webpage
html=requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

#create dataframe with index
df = pd.read_html(html, header = 0)[0]
#ignore cells where Borough is not assigned
df = df[df.Borough != 'Not assigned']
#create table joining multiple neighbourhoods
table = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(list).apply(lambda x:', '.join(x)).to_frame().reset_index()
#use Borough for Neighbourhoods that are not assigned
for index, row in df.iterrows():
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] = row['Borough']

table.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
#determine number of rows and columns 
table.shape

(103, 3)